In [ ]:
# Import libraries
import pandas as pd
import threading
import random
import time
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

# Load the dataset
df = pd.read_csv("Gold Price (2013-2023).csv")

# Fix numeric columns with commas
numeric_columns = ["Price", "Open", "High", "Low"]
for col in numeric_columns:
    df[col] = df[col].str.replace(",", "").astype(float)

# Convert "Vol." column to numeric, handling "K"
df['Vol.'] = df['Vol.'].str.replace("K", "000", regex=True)
df['Vol.'] = df['Vol.'].astype(float)

# Capture the initial state of the DataFrame
before_update = df.copy()

# Start the timer for execution time measurement
start_time = time.time()

# Simulating shared resource access and random modification
random.seed(42)  # Set seed for reproducibility

def increment_price_random():
    global df
    for _ in range(1000):
        idx = random.randint(0, len(df) - 1)  # Random row index
        df.at[idx, "Price"] += 1  # Increment the price of a random row

def decrement_price_random():
    global df
    for _ in range(1000):
        idx = random.randint(0, len(df) - 1)  # Random row index
        df.at[idx, "Price"] -= 1  # Decrement the price of a random row

def increment_volume_random():
    global df
    for _ in range(1000):
        idx = random.randint(0, len(df) - 1)  # Random row index
        df.at[idx, "Vol."] += 10  # Increment the volume of a random row

def decrement_volume_random():
    global df
    for _ in range(1000):
        idx = random.randint(0, len(df) - 1)  # Random row index
        df.at[idx, "Vol."] -= 10  # Decrement the volume of a random row

# Create threads
thread1 = threading.Thread(target=increment_price_random)
thread2 = threading.Thread(target=decrement_price_random)
thread3 = threading.Thread(target=increment_volume_random)
thread4 = threading.Thread(target=decrement_volume_random)

# Start threads
thread1.start()
thread2.start()
thread3.start()
thread4.start()

# Wait for threads to finish
thread1.join()
thread2.join()
thread3.join()
thread4.join()

# End the timer
end_time = time.time()

# Capture the final state of the DataFrame
after_update = df.copy()

# Print the DataFrame before and after the race condition
print("DataFrame BEFORE race condition:")
print(before_update.head())

print("\nDataFrame AFTER race condition:")
print(after_update.head())

# Record execution time
execution_time = end_time - start_time
print(f"\nExecution Time: {execution_time:.4f} seconds")

# Preprocessing for machine learning
X = df[["Open", "High", "Low", "Vol."]]
y = df["Price"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train a Random Forest model
rf_model = RandomForestRegressor(random_state=42, n_estimators=100)
rf_model.fit(X_train_scaled, y_train)

# Make predictions and evaluate
y_pred = rf_model.predict(X_test_scaled)
print("\nR2 Score:", r2_score(y_test, y_pred))
print("Mean Squared Error:", mean_squared_error(y_test, y_pred))


DataFrame BEFORE race condition:
         Date   Price    Open     High      Low    Vol. Change %
0  12/30/2022  1826.2  1821.8  1832.40  1819.80  107.50    0.01%
1  12/29/2022  1826.0  1812.3  1827.30  1811.20  105.99    0.56%
2  12/28/2022  1815.8  1822.4  1822.80  1804.20  118.08   -0.40%
3  12/27/2022  1823.1  1808.2  1841.90  1808.00  159.62    0.74%
4  12/26/2022  1809.7  1805.8  1811.95  1805.55     NaN    0.30%

DataFrame AFTER race condition:
         Date   Price    Open     High      Low    Vol. Change %
0  12/30/2022  1826.2  1821.8  1832.40  1819.80  107.50    0.01%
1  12/29/2022  1827.0  1812.3  1827.30  1811.20  105.99    0.56%
2  12/28/2022  1815.8  1822.4  1822.80  1804.20  128.08   -0.40%
3  12/27/2022  1823.1  1808.2  1841.90  1808.00  159.62    0.74%
4  12/26/2022  1809.7  1805.8  1811.95  1805.55     NaN    0.30%

Execution Time: 0.7353 seconds

R2 Score: 0.999305193278458
Mean Squared Error: 47.382999248870405


In [ ]:

# Critical Section using threading.Lock
lock = threading.Lock()

def increment_price_random():
    global df
    for _ in range(1000):
        idx = random.randint(0, len(df) - 1)
        with lock:  # Critical section
            df.at[idx, "Price"] += 1

def decrement_price_random():
    global df
    for _ in range(1000):
        idx = random.randint(0, len(df) - 1)
        with lock:
            df.at[idx, "Price"] -= 1


In [ ]:

# Reduction Technique
thread_changes = []

def increment_price_random_reduction():
    local_changes = []
    for _ in range(1000):
        idx = random.randint(0, len(df) - 1)
        local_changes.append((idx, 1))
    thread_changes.append(local_changes)

def apply_reduction():
    for changes in thread_changes:
        for idx, change in changes:
            df.at[idx, "Price"] += change


In [ ]:

# Analyzing Performance with Multiple Cores
from multiprocessing import Pool

def worker_function(core_id):
    increment_price_random()

cores = [1, 2, 4, 8]
for core_count in cores:
    start_time = time.time()
    with Pool(core_count) as pool:
        pool.map(worker_function, range(core_count))
    end_time = time.time()
    print(f"Cores: {core_count}, Execution Time: {end_time - start_time:.4f} seconds")


Cores: 1, Execution Time: 0.0822 seconds
Cores: 2, Execution Time: 0.1460 seconds
Cores: 4, Execution Time: 0.2917 seconds
Cores: 8, Execution Time: 0.5613 seconds
